In [ ]:
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from skbio.diversity import beta_diversity
from skbio.stats.ordination import pcoa
from sklearn.decomposition import PCA
import numpy as np

In [ ]:
cog_function = pd.read_table('ref/cog_function.tsv', index_col='cog_id')

In [ ]:
all_frames = []
for path in glob('res/core.a.mags.annot.d/*.cog.tsv') + glob('res/ref.mags.annot.d/*.cog.tsv'):
    frame = pd.read_table(path, names=['orf_id', 'cog'])
    frame['mag_id'] = path.split('/')[-1].split('.')[0]
    all_frames.append(frame)
    
data = (pd.concat(all_frames)
          .reset_index(drop=True)
          .rename(columns={'cog': 'cog_id'})
          .groupby(['mag_id', 'cog_id'])
          .apply(len)
          .unstack('cog_id')
          .fillna(0).astype(int)
       )

In [ ]:
for cog_id in set(cog_function.index) - set(data.columns):
    data[cog_id] = 0

In [ ]:
def pcoa_t(counts, diss='jaccard', return_prop_explained=False):
    dist = beta_diversity(diss, counts.values, ids=counts.index)
    pcoa_fit = pcoa(dist)
    pcoa_result = pcoa_fit.samples
    if return_prop_explained:
        return pcoa_result, pd.Series(pcoa_fit.proportion_explained, index=pcoa_result.columns)
    else:
        return pcoa_result

def pca_t(counts, return_prop_explained=False):
    fit = PCA().fit(counts)
    values = fit.transform(counts)
    out = pd.DataFrame(values, index=counts.index)
    out.rename(lambda i: 'PC{}'.format(i), axis='columns', inplace=True)
    if return_prop_explained:
        return out, pd.Series(fit.explained_variance_ / fit.explained_variance_.sum(), index=out.columns)
    else:
        return out

In [ ]:
ormerod_desig = { 'GP4': 'plant'
                , 'GP3': 'plant'
                , 'H7': 'plant'
                , 'K1': 'plant'
                , 'M13': 'plant'
                , 'GP1': 'plant'
                , 'M2': 'plant'
                , 'M8': 'plant'
                , 'M1': 'plant'
                , 'M12': 'plant'
                , 'H5': 'plant'
                , 'Homeothermus_arabinoxylanisolvens': 'plant'
                , 'GP2': 'host'
                , 'M9': 'host'
                , 'M14': 'host'
                , 'H6': 'host'
                , 'M5': 'host'
                , 'M6': 'starch'
                , 'M11': 'starch'
                , 'H2': 'starch'
                , 'H4': 'starch'
                , 'H10': 'starch'
                , 'M10': 'starch'
                , 'H3': 'starch'
                , 'H8': 'starch'
                , 'H9': 'starch'
                , 'H1': 'starch'
                , 'H3': 'starch'
                , 'M7': 'starch'
                , 'K10': 'starch'
                , 'M3': 'starch'
                }

# Presence Absence of Carbohydrate COGs


In [ ]:
feats = list(set(cog_function[lambda x: x.function_categories.str.contains('G')].index) & set(data.columns))
d = (data[feats]
          .apply(lambda x: (x > 0).astype(int)))

palette = {'starch': 'blue', 'host': 'purple', 'plant': 'green'}


fit = PCA().fit(d)
values = fit.transform(d)
ordin = pd.DataFrame(values, index=d.index).rename(lambda i: 'PC{}'.format(i), axis='columns')
prop_explained = pd.Series(fit.explained_variance_ / fit.explained_variance_.sum(), index=ordin.columns)
ordin, prop_explained = pca_t(np.sqrt(d), return_prop_explained=True)

comps = 'PC1', 'PC2'
fig, ax = plt.subplots(figsize=(12, 12))
ax.scatter(comps[0], comps[1], data=ordin)
for mag_id, row in ordin.iterrows():
    if mag_id in ormerod_desig:
        color = palette[ormerod_desig[mag_id]]
    else:
        color = 'k'
    ax.annotate(mag_id, (row[comps[0]], row[comps[1]]), color=color, )
    ax.set_xlabel('{} ({})'.format(comps[0], prop_explained[comps[0]]))
    ax.set_ylabel('{} ({})'.format(comps[1], prop_explained[comps[1]]))
    
compon = pd.DataFrame(fit.components_.T, index=feats).rename(lambda i: 'PC{}'.format(i), axis='columns')
magnitude = np.sqrt(compon[comps[0]]**2 + compon[comps[1]]**2)
magnitude.name = 'magnitude'
important_compons = compon.loc[magnitude.sort_values(ascending=False).head(10).index]
scale = 5
#ax.scatter(comps[0], comps[1], data=important_compons * scale, color='grey')
for cog_id, row in important_compons.iterrows():
    ax.annotate(cog_id, (row[comps[0]] * scale, row[comps[1]] * scale), color=color, weight='bold')
    
cog_function.join(magnitude).sort_values('magnitude', ascending=False).dropna(subset=['magnitude']).head(10)

# Abundance of Carbohydrate COGs


In [ ]:
feats = list(set(cog_function[lambda x: x.function_categories.str.contains('G')].index) & set(data.columns))
d = (data[feats])

palette = {'starch': 'blue', 'host': 'purple', 'plant': 'green'}


fit = PCA().fit(d)
values = fit.transform(d)
ordin = pd.DataFrame(values, index=d.index).rename(lambda i: 'PC{}'.format(i), axis='columns')
prop_explained = pd.Series(fit.explained_variance_ / fit.explained_variance_.sum(), index=ordin.columns)
ordin, prop_explained = pca_t(np.sqrt(d), return_prop_explained=True)

comps = 'PC1', 'PC2'
fig, ax = plt.subplots(figsize=(12, 12))
ax.scatter(comps[0], comps[1], data=ordin)
for mag_id, row in ordin.iterrows():
    if mag_id in ormerod_desig:
        color = palette[ormerod_desig[mag_id]]
    else:
        color = 'k'
    ax.annotate(mag_id, (row[comps[0]], row[comps[1]]), color=color)
    ax.set_xlabel('{} ({})'.format(comps[0], prop_explained[comps[0]]))
    ax.set_ylabel('{} ({})'.format(comps[1], prop_explained[comps[1]]))
    
compon = pd.DataFrame(fit.components_.T, index=feats).rename(lambda i: 'PC{}'.format(i), axis='columns')
magnitude = np.sqrt(compon[comps[0]]**2 + compon[comps[1]]**2)
magnitude.name = 'magnitude'
important_compons = compon.loc[magnitude.sort_values(ascending=False).head(10).index]
scale = 8
#ax.scatter(comps[0], comps[1], data=important_compons * scale, color='grey')
for cog_id, row in important_compons.iterrows():
    ax.annotate(cog_id, (row[comps[0]] * scale, row[comps[1]] * scale), weight='bold', alpha=0.5)
ax.scatter([0], [0], marker='x', color='k')
    
cog_function.join(magnitude).sort_values('magnitude', ascending=False).dropna(subset=['magnitude']).head(10)

In [ ]:
import seaborn as sns

sns.clustermap(d.loc[:,d.sum() != 0].T.apply(np.sqrt), robust=True,
                   figsize=(10, 10), vmin=0)

# Abundance of Ormerod COGs (mostly carbs)

In [ ]:

feats = list(set(['COG3507', 'COG3866', 'COG4677', 'COG2730', 'COG3693', 'COG0366', 'COG3525', 'COG3119']) &
             set(data.columns))
d = (data[feats])

palette = {'starch': 'blue', 'host': 'purple', 'plant': 'green'}


fit = PCA().fit(d)
values = fit.transform(d)
ordin = pd.DataFrame(values, index=d.index).rename(lambda i: 'PC{}'.format(i), axis='columns')
prop_explained = pd.Series(fit.explained_variance_ / fit.explained_variance_.sum(), index=ordin.columns)
ordin, prop_explained = pca_t(np.sqrt(d), return_prop_explained=True)

comps = 'PC1', 'PC2'
fig, ax = plt.subplots(figsize=(12, 12))
ax.scatter(comps[0], comps[1], data=ordin)
for mag_id, row in ordin.iterrows():
    if mag_id in ormerod_desig:
        color = palette[ormerod_desig[mag_id]]
    else:
        color = 'k'
    ax.annotate(mag_id, (row[comps[0]], row[comps[1]]), color=color)
    ax.set_xlabel('{} ({})'.format(comps[0], prop_explained[comps[0]]))
    ax.set_ylabel('{} ({})'.format(comps[1], prop_explained[comps[1]]))
    
compon = pd.DataFrame(fit.components_.T, index=feats).rename(lambda i: 'PC{}'.format(i), axis='columns')
magnitude = np.sqrt(compon[comps[0]]**2 + compon[comps[1]]**2)
magnitude.name = 'magnitude'
important_compons = compon.loc[magnitude.sort_values(ascending=False).head(10).index]
scale = 2
#ax.scatter(comps[0], comps[1], data=important_compons * scale, color='grey')
for cog_id, row in important_compons.iterrows():
    ax.annotate(cog_id, (row[comps[0]] * scale, row[comps[1]] * scale), color=color, weight='bold')
    
cog_function.join(magnitude).sort_values('magnitude', ascending=False).dropna(subset=['magnitude']).head(10)

# Presence/Absence All COGs

In [ ]:
feats = list(set(cog_function.index) &
             set(data.columns))
d = (data[feats]
         .apply(lambda x: (x > 0).astype(int)))

palette = {'starch': 'blue', 'host': 'purple', 'plant': 'green'}


fit = PCA().fit(d)
values = fit.transform(d)
ordin = pd.DataFrame(values, index=d.index).rename(lambda i: 'PC{}'.format(i), axis='columns')
prop_explained = pd.Series(fit.explained_variance_ / fit.explained_variance_.sum(), index=ordin.columns)
ordin, prop_explained = pca_t(np.sqrt(d), return_prop_explained=True)

comps = 'PC1', 'PC2'
fig, ax = plt.subplots(figsize=(12, 12))
ax.scatter(comps[0], comps[1], data=ordin)
for mag_id, row in ordin.iterrows():
    if mag_id in ormerod_desig:
        color = palette[ormerod_desig[mag_id]]
    else:
        color = 'k'
    ax.annotate(mag_id, (row[comps[0]], row[comps[1]]), color=color)
    ax.set_xlabel('{} ({})'.format(comps[0], prop_explained[comps[0]]))
    ax.set_ylabel('{} ({})'.format(comps[1], prop_explained[comps[1]]))
    
compon = pd.DataFrame(fit.components_.T, index=feats).rename(lambda i: 'PC{}'.format(i), axis='columns')
magnitude = np.sqrt(compon[comps[0]]**2 + compon[comps[1]]**2)
magnitude.name = 'magnitude'
important_compons = compon.loc[magnitude.sort_values(ascending=False).head(10).index]
scale = 30
#ax.scatter(comps[0], comps[1], data=important_compons * scale, color='grey')
for cog_id, row in important_compons.iterrows():
    ax.annotate(cog_id, (row[comps[0]] * scale, row[comps[1]] * scale), color=color)
    
cog_function.join(magnitude).sort_values('magnitude', ascending=False).dropna(subset=['magnitude']).head(10)

# Frequency of COGs present in both OTU1 and OTU4 in Ormerod Strains


In [ ]:
freq_in_ormerod_strains = (data.loc[data.index.str.contains('^(M|GP|H|K)[0-9]')] > 0).mean()
freq_in_ormerod_strains.name = 'freq'


(cog_function.loc[(data.loc['OTU-1-UM'] > 0) &
                  (data.loc['OTU-7'] > 0)]
             .join(freq_in_ormerod_strains)
             .sort_values('freq')
             
).head(20)#[lambda x: x.function_categories.str.contains('G')]

# COGs present in OTU-1-UM and OTU-7, but not in OTU-1-UT


In [ ]:
freq_in_ormerod_strains = (data.loc[data.index.str.contains('^(M|GP|H|K)[0-9]')] > 0).mean()
freq_in_ormerod_strains.name = 'freq'

(cog_function.loc[(data.loc['OTU-1-UM'] > 0) &
                  (data.loc['OTU-7'] > 0) &
                  (data.loc['OTU-1-UT'] == 0)]
            .join(freq_in_ormerod_strains)).sort_values('freq')  #[lambda x: x.function_categories.str.contains('G')]

# COGs present in OTU-1-UT and OTU-7, but not in OTU-1-UM

In [ ]:
freq_in_ormerod_strains = (data.loc[data.index.str.contains('^(M|GP|H|K)[0-9]')] > 0).mean()
freq_in_ormerod_strains.name = 'freq'

(cog_function.loc[(data.loc['OTU-1-UT'] > 0) &
                  (data.loc['OTU-7'] > 0) &
                  (data.loc['OTU-1-UM'] == 0)]
            .join(freq_in_ormerod_strains)).sort_values('freq')  #[lambda x: x.function_categories.str.contains('G')]

# COGs in OTU-1 (both sites) but not OTU-7

In [ ]:
freq_in_ormerod_strains = (data.loc[data.index.str.contains('^(M|GP|H|K)[0-9]')] > 0).mean()
freq_in_ormerod_strains.name = 'freq'

(cog_function.loc[((data.loc['OTU-1-UT'] + data.loc['OTU-1-UM']) > 0) &
                  (data.loc['OTU-7'] == 0)]
             .join(freq_in_ormerod_strains)).sort_values('freq')  #[lambda x: x.function_categories.str.contains('G')]

In [ ]:
freq_in_ormerod_strains = (data.loc[data.index.str.contains('^(M|GP|H|K)[0-9]')] > 0).mean()
freq_in_ormerod_strains.name = 'freq'

(cog_function.loc[(data.loc['OTU-7'] > 0) &
                  ((data.loc['OTU-1-UT'] + data.loc['OTU-1-UM']) == 0)
                 ]
             .join(freq_in_ormerod_strains)).sort_values('freq')  #[lambda x: x.function_categories.str.contains('G')]

# MinPath to MetaCyc pathways

In [ ]:
metacyc_meta = pd.read_table('ref/metacyc_pathway_descriptions.tsv', names=['metacyc_id', 'description'], index_col='metacyc_id')

In [ ]:
from itertools import chain

rawdata = {}

for filepath in chain(glob('res/core.a.mags.annot.d/*.minpath.list'),
                      glob('res/ref.mags.annot.d/*.minpath.list')):
    with open(filepath) as handle:
        genome_name = filepath.split('/')[-1].split('.')[0]
        rawdata[genome_name] = {}
        for pathname in (line.strip() for line in handle):
            rawdata[genome_name][pathname] = 1
            
pathdata = pd.DataFrame(rawdata).fillna(0).astype(int).T
pathdata.index.name = 'mag_id'
pathdata.columns.name = 'metacyc_id'

In [ ]:
pathdata.sum(1).sort_values()

In [ ]:
d = pathdata
feats = d.columns

palette = {'starch': 'blue', 'host': 'purple', 'plant': 'green'}


fit = PCA().fit(d)
values = fit.transform(d)
ordin = pd.DataFrame(values, index=d.index).rename(lambda i: 'PC{}'.format(i), axis='columns')
prop_explained = pd.Series(fit.explained_variance_ / fit.explained_variance_.sum(), index=ordin.columns)
ordin, prop_explained = pcoa_t(d, return_prop_explained=True)

comps = 'PC1', 'PC2'
fig, ax = plt.subplots(figsize=(12, 12))
ax.scatter(comps[0], comps[1], data=ordin)
for mag_id, row in ordin.iterrows():
    if mag_id in ormerod_desig:
        color = palette[ormerod_desig[mag_id]]
    else:
        color = 'k'
    ax.annotate(mag_id, (row[comps[0]], row[comps[1]]), color=color)
    ax.set_xlabel('{} ({})'.format(comps[0], prop_explained[comps[0]]))
    ax.set_ylabel('{} ({})'.format(comps[1], prop_explained[comps[1]]))
    
compon = pd.DataFrame(fit.components_.T, index=feats).rename(lambda i: 'PC{}'.format(i), axis='columns')
magnitude = np.sqrt(compon[comps[0]]**2 + compon[comps[1]]**2)
magnitude.name = 'magnitude'
important_compons = compon.loc[magnitude.sort_values(ascending=False).head(10).index]
    
#cog_function.join(magnitude).sort_values('magnitude', ascending=False).dropna(subset=['magnitude']).head(10)

In [ ]:
pathdata.loc[['OTU-1-UT', 'OTU-1-UM', 'OTU-7']].T[lambda x: (x['OTU-1-UM'] & x['OTU-7'] & ~x['OTU-1-UT']).astype(bool)].join(metacyc_meta)

In [ ]:
sns.clustermap(pathdata.loc[:,~(pathdata.all() | ~pathdata.any())])

In [ ]:
pathdata.sum(1).sort_values()

In [ ]:
freq_in_ormerod_strain = (pathdata.loc[pathdata.index.str.contains('^(M|GP|H|K)[0-9]')] > 0).mean()
(freq_in_ormerod_strain[pathdata.T[lambda x: (x['OTU-1-UM'] & ~x['OTU-1-UT'])
                                .astype(bool)].index]
                       .sort_values().to_frame(name='freq').join(metacyc_meta))

In [ ]:
freq_in_ormerod_strain = (pathdata.loc[pathdata.index.str.contains('^(M|GP|H|K)[0-9]')] > 0).mean()
(freq_in_ormerod_strain[pathdata.T[lambda x: (x['OTU-1-UM'] & x['OTU-7'])
                                .astype(bool)].index]
                       .to_frame(name='freq').join(metacyc_meta)
                       .sort_values('freq').head(15))